In [1]:
from lxml import html  
import csv,os,json
import requests
import pandas as pd
from time import sleep

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

In [3]:
# keywords="biomedical+material"
# search_url="https://www.scientificamerican.com/search/?q="+keywords
# next_page=get_next_page(search_url)
# print(next_page)
# # executable_path = 'C_driver/geckodriver'
# driver = webdriver.Firefox(executable_path=executable_path)
# main_search=driver.get(next_page)

In [28]:
def get_next_page(search_url):
    
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}
    page = requests.get(search_url,headers=headers)
    sleep(2)
    tree = html.fromstring(page.content)
    
    next_page_xpath="//a[@class='pagination__prev']/@href"
    next_page_url=tree.xpath(next_page_xpath)
#     print(next_page_url)
#     next_page_url='https://www.scientificamerican.com/search/'+''.join(tree.xpath(next_page_xpath))
    
    return next_page_url

if __name__ == "__main__":
    
#     keywords="biomedical+material"
    keyword="tissue engineering"
    item=keyword.split(" ")
    search_url="https://www.scientificamerican.com/search/?q=biomaterial%2C+biomedical+material%2C+"+item[0]+"+"+item[1]
#     keywords=["microstructure","adhesion","scaffold","polymer","regeneration","nanoparticle","hydrogel","chitosan","bone"]
#     url_list=[]
#     for item in keywords:
        
#         search_url="https://www.scientificamerican.com/search/?q=biomaterial%2C+biomedical+material%2C+"+item
#         url_list.append(search_url)
        
# https://www.scientificamerican.com/search/?q=biomaterial%2C+biomedical+material%2C+microstructure
# https://www.scientificamerican.com/search/?q=biomaterial%2C+biomedical+material%2C+adhesion
#     print(url_list)

    executable_path = 'C_driver/geckodriver'
    driver = webdriver.Firefox(executable_path=executable_path)
#     main_search=driver.get(url_list[8])
    main_search=driver.get(search_url)
    
    res_list=[]
    num_res_css="div.container.search__filters p"
    num_res=driver.find_element_by_css_selector(num_res_css)
    res_list.append(num_res.text)
    print(res_list)

    results=driver.find_elements_by_css_selector("[class=listing-wide]")
    data_res=[]
    for result in results:
        try: title=result.find_element_by_css_selector("[class=t_listing-title]").text
        except: print ('no title') 
    
        try: date=result.find_element_by_css_selector("[class=t_meta]").text
        except: print('no text')
    
        data={
            'Title':title,
            'Date':date
        }
        data_res.append(data)

    i=2
    data_next=[]
#     temp=url_list[8]
    while i<=31:
        
        print('page_number:'+' '+str(i))
        
#         next_page_url=get_next_page(temp)
        next_page_url=get_next_page(search_url)
        print(next_page_url)
        if len(next_page_url)==1: 
            next_page_url='https://www.scientificamerican.com/search/'+''.join(next_page_url)
        else:
            next_page_url='https://www.scientificamerican.com/search/'+''.join(next_page_url[1])
        
        print(next_page_url)
        main_search=driver.get(next_page_url)
        sleep(3)
        results=driver.find_elements_by_css_selector("[class=listing-wide]")
        
        for result in results:
            try: title=result.find_element_by_css_selector("[class=t_listing-title]").text
            except: print ('no title') 
    
            try: date=result.find_element_by_css_selector("[class=t_meta]").text
            except: print('no text')
    
            data={
                'Title':title,
                'Date':date
            }
            data_next.append(data)
        
#         print(len(data_next))
        i=i+1
#         temp=next_page_url
        search_url=next_page_url
    
    data_all=data_res+data_next
    
#     print('Length of all data:'+str(len(data_all)))
    driver.quit()
    
    with open('tissue.json', 'w') as outfile:
        json.dump(data_all, outfile)

['Your search found 603 results']
page_number: 2
['?page=2&q=biomaterial, biomedical material, tissue engineering']
https://www.scientificamerican.com/search/?page=2&q=biomaterial, biomedical material, tissue engineering
page_number: 3
['?page=1&q=biomaterial, biomedical material, tissue engineering', '?page=3&q=biomaterial, biomedical material, tissue engineering']
https://www.scientificamerican.com/search/?page=3&q=biomaterial, biomedical material, tissue engineering
page_number: 4
['?page=2&q=biomaterial, biomedical material, tissue engineering', '?page=4&q=biomaterial, biomedical material, tissue engineering']
https://www.scientificamerican.com/search/?page=4&q=biomaterial, biomedical material, tissue engineering
page_number: 5
['?page=3&q=biomaterial, biomedical material, tissue engineering', '?page=5&q=biomaterial, biomedical material, tissue engineering']
https://www.scientificamerican.com/search/?page=5&q=biomaterial, biomedical material, tissue engineering
page_number: 6
['?pa

In [29]:
data_df = pd.read_json('tissue.json')
data_df.tail()

,Date,Title
598,"November 1, 2008 — Bernhard Blümich",The Incredible Shrinking Scanner: MRI-like Mac...
599,"February 1, 2009 — James R. Heath, Mark E. Dav...",Nanomedicine--Revolutionizing the Fight agains...
600,"April 1, 1930","Sinking a ""Mine"" Under New York City, ""Spider ..."
601,"January 1, 2010 — John D. Young and Jan Martel",The Truth about Nanobacteria
602,"February 1, 2014 — Rakesh K. Jain",Viewing Cancer as a Physics Problem Suggests N...


In [27]:
keyword="tissue engineering"
keyword.split(" ")

['tissue', 'engineering']